## Preprocessing

- 윈도우 안에서 각각 normalize 

In [721]:
import pandas as pd
import pandas_ta as ta

일별 데이터에서 quote_av라는 컬럼이 있으면, 이는 그 날의 평균 거래량을 나타낼 수 있으며, 주간 데이터에서는 해당 주의 평균 거래량을 나타낼 수 있습니다.

In [722]:
btc = pd.read_csv("dataset/BTC_USDT_1d_All.csv")

btc

,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,Symbol
0,2017-08-17 09:00:00,4261.48,4485.39,4200.74,4285.08,795.150377,3.454770e+06,3427,616.248541,2.678216e+06,BTCUSDT
1,2017-08-18 09:00:00,4285.08,4371.52,3938.77,4108.37,1199.888264,5.086958e+06,5233,972.868710,4.129123e+06,BTCUSDT
2,2017-08-19 09:00:00,4108.37,4184.69,3850.00,4139.98,381.309763,1.549484e+06,2153,274.336042,1.118002e+06,BTCUSDT
3,2017-08-20 09:00:00,4120.98,4211.08,4032.62,4086.29,467.083022,1.930364e+06,2321,376.795947,1.557401e+06,BTCUSDT
4,2017-08-21 09:00:00,4069.13,4119.62,3911.79,4016.00,691.743060,2.797232e+06,3972,557.356107,2.255663e+06,BTCUSDT
...,...,...,...,...,...,...,...,...,...,...,...
2224,2023-09-19 09:00:00,26762.50,27483.57,26667.79,27210.26,36190.521870,9.815393e+08,898076,17851.155580,4.840613e+08,BTCUSDT
2225,2023-09-20 09:00:00,27210.25,27388.63,26800.00,27125.00,34207.218670,9.272061e+08,867288,16983.430860,4.604023e+08,BTCUSDT
2226,2023-09-21 09:00:00,27125.01,27159.60,26377.70,26568.08,34476.826620,9.207681e+08,823416,16878.782020,4.507384e+08,BTCUSDT
2227,2023-09-22 09:00:00,26568.08,26743.38,26468.77,26580.14,18198.229200,4.843316e+08,567149,8484.096790,2.258057e+08,BTCUSDT


In [723]:
# 데이터 파일 읽기
vix = pd.read_csv("dataset/VIX_History.csv")


In [724]:
# 날짜 형식 맞추기
btc['Open_time'] = pd.to_datetime(btc['Open_time']).dt.date
vix['DATE'] = pd.to_datetime(vix['DATE']).dt.date

# VIX 데이터에 누락된 날짜 채우기
all_dates = pd.date_range(start=min(vix['DATE']), end=max(vix['DATE'])).date
vix_full = pd.DataFrame({'DATE': all_dates}).merge(vix, on='DATE', how='left')

# 누락된 VIX 값에 대해 나중의 값으로 대체
vix_full['CLOSE'].fillna(method='bfill', inplace=True)

# 'Open_time'과 'DATE' 기준으로 두 데이터프레임 병합
merged = pd.merge(btc, vix_full[['DATE', 'CLOSE']], left_on='Open_time', right_on='DATE', how='inner')

# 'DATE' 컬럼은 중복이므로 삭제
merged.drop(columns=['DATE'], inplace=True)

# 'Symbol' 컬럼 삭제 (만약 btc 데이터프레임에 'Symbol'이 있을 경우)
if 'Symbol' in merged.columns:
    merged.drop(columns=['Symbol'], inplace=True)

# 'CLOSE' 컬럼의 이름을 'VIX'로 변경
merged.rename(columns={'CLOSE': 'VIX'}, inplace=True)

# 결과 확인
merged


,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,VIX
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,3.454770e+06,3427,616.248541,2.678216e+06,15.55
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,5.086958e+06,5233,972.868710,4.129123e+06,14.26
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,1.549484e+06,2153,274.336042,1.118002e+06,13.19
3,2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,1.930364e+06,2321,376.795947,1.557401e+06,13.19
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,2.797232e+06,3972,557.356107,2.255663e+06,13.19
...,...,...,...,...,...,...,...,...,...,...,...
2216,2023-09-11,25841.60,25900.69,24901.00,25162.52,41682.320000,1.057372e+09,864138,19664.902430,4.988216e+08,13.80
2217,2023-09-12,25162.53,26567.00,25131.48,25840.10,56434.385370,1.464393e+09,1047056,27720.518300,7.191473e+08,14.23
2218,2023-09-13,25840.10,26405.22,25764.17,26222.00,31610.827530,8.254379e+08,714880,15287.000380,3.990998e+08,13.48
2219,2023-09-14,26222.00,26860.49,26126.77,26522.73,38333.172500,1.015267e+09,799278,19440.454670,5.148638e+08,12.82


In [725]:
btc_fee = pd.read_csv('dataset/BitcoinTransactionFee.csv')

In [726]:
# 날짜 형식 통일
btc_fee['Date'] = pd.to_datetime(btc_fee['Date']).dt.date

# 기존의 병합된 데이터프레임과 날짜(date) 기준으로 병합
feeadded = pd.merge(merged, btc_fee[['Date', 'Value']], left_on='Open_time', right_on='Date', how='inner')

# 'Date' 열 제거하고 'Value' 열의 이름을 'Bitcoin_TransactionFee'로 변경
feeadded.drop('Date', axis=1, inplace=True)
feeadded.rename(columns={'Value': 'Bitcoin_TransactionFee'}, inplace=True)

# 최종 데이터프레임 확인
feeadded


,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,VIX,Bitcoin_TransactionFee
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,3.454770e+06,3427,616.248541,2.678216e+06,15.55,365.316349
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,5.086958e+06,5233,972.868710,4.129123e+06,14.26,337.961923
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,1.549484e+06,2153,274.336042,1.118002e+06,13.19,265.911799
3,2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,1.930364e+06,2321,376.795947,1.557401e+06,13.19,208.981996
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,2.797232e+06,3972,557.356107,2.255663e+06,13.19,297.964965
...,...,...,...,...,...,...,...,...,...,...,...,...
2216,2023-09-11,25841.60,25900.69,24901.00,25162.52,41682.320000,1.057372e+09,864138,19664.902430,4.988216e+08,13.80,32.697255
2217,2023-09-12,25162.53,26567.00,25131.48,25840.10,56434.385370,1.464393e+09,1047056,27720.518300,7.191473e+08,14.23,31.029445
2218,2023-09-13,25840.10,26405.22,25764.17,26222.00,31610.827530,8.254379e+08,714880,15287.000380,3.990998e+08,13.48,31.128702
2219,2023-09-14,26222.00,26860.49,26126.77,26522.73,38333.172500,1.015267e+09,799278,19440.454670,5.148638e+08,12.82,30.897714


In [727]:
# voldata.csv 파일 읽기
vol = pd.read_csv('dataset/voldata.csv')

In [728]:
import pandas as pd

# 예시 데이터 (실제 데이터를 사용해주세요)
# feeadded = ...
# vol = ...

# 날짜 형식 통일
vol['Open_time'] = pd.to_datetime(vol['Open_time']).dt.date

# vol 데이터에 누락된 날짜 채우기
all_dates = pd.date_range(start=min(vol['Open_time']), end=max(vol['Open_time'])).date
vol_full = pd.DataFrame({'Open_time': all_dates}).merge(vol, on='Open_time', how='left')

# 누락된 vol 값에 대해 나중의 값으로 대체
# NOTE: 여기서 'COLUMN_NAME'은 대체해야 할 vol의 컬럼 이름으로 교체해주세요
vol_full['vol'].fillna(method='bfill', inplace=True)

# 기존의 병합된 데이터프레임과 날짜(date) 기준으로 병합
voladded = pd.merge(feeadded, vol_full, on='Open_time', how='inner')

# 최종 데이터프레임 확인
voladded


,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,VIX,Bitcoin_TransactionFee,Unnamed: 0,price,rate,vol
0,2017-09-16,3674.01,3950.00,3470.66,3714.95,1297.563953,4.804878e+06,8809,624.887842,2.314536e+06,10.15,119.917805,30,3714.95,0.004041,1.151416
1,2017-09-17,3685.23,3748.21,3499.02,3699.99,682.171210,2.466490e+06,6355,273.931976,9.948436e+05,10.15,99.410660,31,3699.99,-0.004027,1.143121
2,2017-09-18,3690.00,4123.20,3690.00,4035.01,1030.006455,4.069033e+06,8220,417.638722,1.651863e+06,10.15,168.308100,32,4035.01,0.090546,1.187358
3,2017-09-19,4060.00,4089.97,3830.91,3910.04,902.332129,3.558315e+06,7342,257.320352,1.016804e+06,10.18,173.703415,33,3910.04,-0.030971,1.191689
4,2017-09-20,3910.04,4046.08,3820.00,3900.00,720.935076,2.839584e+06,7231,395.275138,1.562210e+06,9.78,146.407010,34,3900.00,-0.002568,1.190050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,2023-09-11,25841.60,25900.69,24901.00,25162.52,41682.320000,1.057372e+09,864138,19664.902430,4.988216e+08,13.80,32.697255,2216,25162.52,-0.026279,0.413330
2187,2023-09-12,25162.53,26567.00,25131.48,25840.10,56434.385370,1.464393e+09,1047056,27720.518300,7.191473e+08,14.23,31.029445,2217,25840.10,0.026928,0.428066
2188,2023-09-13,25840.10,26405.22,25764.17,26222.00,31610.827530,8.254379e+08,714880,15287.000380,3.990998e+08,13.48,31.128702,2218,26222.00,0.014779,0.432137
2189,2023-09-14,26222.00,26860.49,26126.77,26522.73,38333.172500,1.015267e+09,799278,19440.454670,5.148638e+08,12.82,30.897714,2219,26522.73,0.011469,0.434986


In [729]:
# 'Unnamed: 0', 'price', 'rate' 열 제거
voladded.drop(['Unnamed: 0', 'price', 'rate'], axis=1, inplace=True)

# 'vol' 열의 이름을 'volatility'로 변경
voladded.rename(columns={'vol': 'volatility'}, inplace=True)

# 변경된 데이터프레임 확인
voladded


,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,VIX,Bitcoin_TransactionFee,volatility
0,2017-09-16,3674.01,3950.00,3470.66,3714.95,1297.563953,4.804878e+06,8809,624.887842,2.314536e+06,10.15,119.917805,1.151416
1,2017-09-17,3685.23,3748.21,3499.02,3699.99,682.171210,2.466490e+06,6355,273.931976,9.948436e+05,10.15,99.410660,1.143121
2,2017-09-18,3690.00,4123.20,3690.00,4035.01,1030.006455,4.069033e+06,8220,417.638722,1.651863e+06,10.15,168.308100,1.187358
3,2017-09-19,4060.00,4089.97,3830.91,3910.04,902.332129,3.558315e+06,7342,257.320352,1.016804e+06,10.18,173.703415,1.191689
4,2017-09-20,3910.04,4046.08,3820.00,3900.00,720.935076,2.839584e+06,7231,395.275138,1.562210e+06,9.78,146.407010,1.190050
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,2023-09-11,25841.60,25900.69,24901.00,25162.52,41682.320000,1.057372e+09,864138,19664.902430,4.988216e+08,13.80,32.697255,0.413330
2187,2023-09-12,25162.53,26567.00,25131.48,25840.10,56434.385370,1.464393e+09,1047056,27720.518300,7.191473e+08,14.23,31.029445,0.428066
2188,2023-09-13,25840.10,26405.22,25764.17,26222.00,31610.827530,8.254379e+08,714880,15287.000380,3.990998e+08,13.48,31.128702,0.432137
2189,2023-09-14,26222.00,26860.49,26126.77,26522.73,38333.172500,1.015267e+09,799278,19440.454670,5.148638e+08,12.82,30.897714,0.434986


In [730]:
googletrend = pd.read_csv('dataset/google trend data.csv')


In [731]:
import pandas as pd

# 날짜 형식 통일
googletrend['Date'] = pd.to_datetime(googletrend['Date']).dt.date

# googletrend 데이터에 누락된 날짜 채우기
all_dates = pd.date_range(start=min(googletrend['Date']), end=max(googletrend['Date'])).date
gt_full = pd.DataFrame({'Date': all_dates}).merge(googletrend, on='Date', how='left')

# 누락된 googletrend 값에 대해 나중의 값으로 대체
# NOTE: 여기서 'COLUMN_NAME'은 대체해야 할 googletrend의 컬럼 이름으로 교체해주세요
gt_full['GoogleTrend'].fillna(method='bfill', inplace=True)

# 기존의 병합된 데이터프레임과 날짜(date) 기준으로 병합
gtadded = pd.merge(voladded, gt_full, left_on='Open_time', right_on='Date', how='inner')

# 중복된 'Date' 열 제거
gtadded.drop('Date', axis=1, inplace=True)

# 최종 데이터프레임 확인
gtadded


,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend
0,2018-01-01,13715.65,13818.55,12750.00,13380.00,8609.915844,1.147997e+08,105595,3961.938946,5.280975e+07,9.77,469.413036,1.588163,45
1,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,9.77,649.164976,1.616095,53
2,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,9.15,749.685473,1.613762,55
3,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,9.22,778.261095,1.613778,54
4,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,9.22,701.356021,1.580870,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064,2023-08-27,26017.38,26182.23,25966.11,26101.77,12099.642160,3.155484e+08,349090,6170.486640,1.609182e+08,15.08,13.471949,0.300414,45
2065,2023-08-28,26101.78,26253.99,25864.50,26120.00,22692.626550,5.910089e+08,523057,10912.405490,2.842262e+08,15.08,16.972956,0.300290,50
2066,2023-08-29,26120.00,28142.85,25922.00,27716.34,74251.994880,2.023326e+09,1137083,37945.550710,1.034244e+09,14.45,21.151151,0.376069,64
2067,2023-08-30,27716.34,27768.57,27017.24,27299.99,35448.258480,9.690865e+08,690292,16715.577120,4.569264e+08,13.88,19.220988,0.378955,55


In [732]:
# mining_difficulty.csv 파일 읽기
DJ = pd.read_csv('dataset/Dow Jones Industrial Average Historical Data.csv')

In [733]:
import pandas as pd

# DJ 데이터의 날짜 형식 통일
DJ['Date'] = pd.to_datetime(DJ['Date']).dt.date

# DJ 데이터에 누락된 날짜 채우기
all_dates = pd.date_range(start=min(DJ['Date']), end=max(DJ['Date'])).date
dj_full = pd.DataFrame({'Date': all_dates}).merge(DJ, on='Date', how='left')

# 누락된 DJ 값에 대해 나중의 값으로 대체
dj_full['Price'].fillna(method='bfill', inplace=True)

# 기존의 병합된 데이터프레임과 날짜(date) 기준으로 병합
djadded = pd.merge(gtadded, dj_full[['Date', 'Price']], left_on='Open_time', right_on='Date', how='inner')

# 중복된 'Date' 열 제거
djadded.drop('Date', axis=1, inplace=True)

# 'Price' 열의 이름을 'DowJones'로 변경
djadded.rename(columns={'Price': 'DowJones'}, inplace=True)

# 변경된 데이터프레임 확인
djadded


,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones
0,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,9.77,649.164976,1.616095,53,"24,824.01"
1,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,9.15,749.685473,1.613762,55,"24,922.68"
2,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,9.22,778.261095,1.613778,54,"25,075.13"
3,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,9.22,701.356021,1.580870,52,"25,295.87"
4,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,3.092169e+08,158242,11007.164056,1.834178e+08,9.52,694.477906,1.378919,49,"25,283.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,2023-08-27,26017.38,26182.23,25966.11,26101.77,12099.642160,3.155484e+08,349090,6170.486640,1.609182e+08,15.08,13.471949,0.300414,45,"34,560.04"
2064,2023-08-28,26101.78,26253.99,25864.50,26120.00,22692.626550,5.910089e+08,523057,10912.405490,2.842262e+08,15.08,16.972956,0.300290,50,"34,560.04"
2065,2023-08-29,26120.00,28142.85,25922.00,27716.34,74251.994880,2.023326e+09,1137083,37945.550710,1.034244e+09,14.45,21.151151,0.376069,64,"34,852.87"
2066,2023-08-30,27716.34,27768.57,27017.24,27299.99,35448.258480,9.690865e+08,690292,16715.577120,4.569264e+08,13.88,19.220988,0.378955,55,"34,890.77"


In [734]:
# mining_difficulty.csv 파일 읽기
ES = pd.read_csv('dataset/Euro Stoxx 50 Historical Data.csv')

In [735]:
import pandas as pd

# ES 데이터의 날짜 형식 통일
ES['Date'] = pd.to_datetime(ES['Date']).dt.date

# ES 데이터에 누락된 날짜 채우기
all_dates = pd.date_range(start=min(ES['Date']), end=max(ES['Date'])).date
es_full = pd.DataFrame({'Date': all_dates}).merge(ES, on='Date', how='left')

# 누락된 ES 값에 대해 나중의 값으로 대체
es_full['Price'].fillna(method='bfill', inplace=True)

# 기존의 병합된 데이터프레임과 날짜(date) 기준으로 병합
esadded = pd.merge(djadded, es_full[['Date', 'Price']], left_on='Open_time', right_on='Date', how='inner')

# 중복된 'Date' 열 제거
esadded.drop('Date', axis=1, inplace=True)

# 'Price' 열의 이름을 'EuroStoxx50'로 변경
esadded.rename(columns={'Price': 'EuroStoxx50'}, inplace=True)

# 변경된 데이터프레임 확인
esadded


,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50
0,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,9.77,649.164976,1.616095,53,"24,824.01","3,490.19"
1,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,9.15,749.685473,1.613762,55,"24,922.68","3,509.88"
2,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,9.22,778.261095,1.613778,54,"25,075.13","3,568.88"
3,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,9.22,701.356021,1.580870,52,"25,295.87","3,607.63"
4,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,3.092169e+08,158242,11007.164056,1.834178e+08,9.52,694.477906,1.378919,49,"25,283.00","3,616.45"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,2023-08-27,26017.38,26182.23,25966.11,26101.77,12099.642160,3.155484e+08,349090,6170.486640,1.609182e+08,15.08,13.471949,0.300414,45,"34,560.04","4,293.69"
2064,2023-08-28,26101.78,26253.99,25864.50,26120.00,22692.626550,5.910089e+08,523057,10912.405490,2.842262e+08,15.08,16.972956,0.300290,50,"34,560.04","4,293.69"
2065,2023-08-29,26120.00,28142.85,25922.00,27716.34,74251.994880,2.023326e+09,1137083,37945.550710,1.034244e+09,14.45,21.151151,0.376069,64,"34,852.87","4,326.47"
2066,2023-08-30,27716.34,27768.57,27017.24,27299.99,35448.258480,9.690865e+08,690292,16715.577120,4.569264e+08,13.88,19.220988,0.378955,55,"34,890.77","4,315.31"


In [736]:
# mining_difficulty.csv 파일 읽기
nasdaq= pd.read_excel('dataset/Nasdaq index data.xlsx')

In [737]:
import pandas as pd

# 예시 데이터 (실제 데이터를 사용해주세요)
# esadded = ...
# nasdaq = ...

# nasdaq 데이터의 날짜 형식 통일
nasdaq['Date'] = pd.to_datetime(nasdaq['Date']).dt.date

# nasdaq 데이터에 누락된 날짜 채우기
all_dates = pd.date_range(start=min(nasdaq['Date']), end=max(nasdaq['Date'])).date
nd_full = pd.DataFrame({'Date': all_dates}).merge(nasdaq, on='Date', how='left')

# 누락된 nasdaq 값에 대해 나중의 값으로 대체
nd_full['Close/Last'].fillna(method='bfill', inplace=True)

# 기존의 병합된 데이터프레임과 날짜(date) 기준으로 병합
ndadded = pd.merge(esadded, nd_full[['Date', 'Close/Last']], left_on='Open_time', right_on='Date', how='inner')

# 중복된 'Date' 열 제거
ndadded.drop('Date', axis=1, inplace=True)

# 'Close/Last' 열의 이름을 'Nasdaq'로 변경
ndadded.rename(columns={'Close/Last': 'Nasdaq'}, inplace=True)

# 변경된 데이터프레임 확인
ndadded


,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,Nasdaq
0,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,9.77,649.164976,1.616095,53,"24,824.01","3,490.19",7006.90
1,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,9.15,749.685473,1.613762,55,"24,922.68","3,509.88",7065.53
2,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,9.22,778.261095,1.613778,54,"25,075.13","3,568.88",7077.91
3,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,9.22,701.356021,1.580870,52,"25,295.87","3,607.63",7136.56
4,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,3.092169e+08,158242,11007.164056,1.834178e+08,9.52,694.477906,1.378919,49,"25,283.00","3,616.45",7157.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,2023-08-27,26017.38,26182.23,25966.11,26101.77,12099.642160,3.155484e+08,349090,6170.486640,1.609182e+08,15.08,13.471949,0.300414,45,"34,560.04","4,293.69",13705.13
2064,2023-08-28,26101.78,26253.99,25864.50,26120.00,22692.626550,5.910089e+08,523057,10912.405490,2.842262e+08,15.08,16.972956,0.300290,50,"34,560.04","4,293.69",13705.13
2065,2023-08-29,26120.00,28142.85,25922.00,27716.34,74251.994880,2.023326e+09,1137083,37945.550710,1.034244e+09,14.45,21.151151,0.376069,64,"34,852.87","4,326.47",13943.76
2066,2023-08-30,27716.34,27768.57,27017.24,27299.99,35448.258480,9.690865e+08,690292,16715.577120,4.569264e+08,13.88,19.220988,0.378955,55,"34,890.77","4,315.31",14019.31


In [738]:
oil = pd.read_csv('dataset/Oil WTI Futures Historical Data.csv')

In [739]:
import pandas as pd

# 예시 데이터 (실제 데이터를 사용해주세요)
# ndadded = ...
# oil = ...

# oil 데이터의 날짜 형식 통일
oil['Date'] = pd.to_datetime(oil['Date']).dt.date

# oil 데이터에 누락된 날짜 채우기
all_dates = pd.date_range(start=min(oil['Date']), end=max(oil['Date'])).date
oil_full = pd.DataFrame({'Date': all_dates}).merge(oil, on='Date', how='left')

# 누락된 oil 값에 대해 나중의 값으로 대체
oil_full['Price'].fillna(method='bfill', inplace=True)

# 기존의 병합된 데이터프레임과 날짜(date) 기준으로 병합
oiladded = pd.merge(ndadded, oil_full[['Date', 'Price']], left_on='Open_time', right_on='Date', how='inner')

# 중복된 'Date' 열 제거
oiladded.drop('Date', axis=1, inplace=True)

# 'Price' 열의 이름을 'OilWTI'로 변경
oiladded.rename(columns={'Price': 'OilWTI'}, inplace=True)

# 변경된 데이터프레임 확인
oiladded


,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,Nasdaq,OilWTI
0,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,9.77,649.164976,1.616095,53,"24,824.01","3,490.19",7006.90,60.37
1,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,9.15,749.685473,1.613762,55,"24,922.68","3,509.88",7065.53,61.63
2,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,9.22,778.261095,1.613778,54,"25,075.13","3,568.88",7077.91,62.01
3,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,9.22,701.356021,1.580870,52,"25,295.87","3,607.63",7136.56,61.44
4,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,3.092169e+08,158242,11007.164056,1.834178e+08,9.52,694.477906,1.378919,49,"25,283.00","3,616.45",7157.39,61.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,2023-08-27,26017.38,26182.23,25966.11,26101.77,12099.642160,3.155484e+08,349090,6170.486640,1.609182e+08,15.08,13.471949,0.300414,45,"34,560.04","4,293.69",13705.13,80.10
2064,2023-08-28,26101.78,26253.99,25864.50,26120.00,22692.626550,5.910089e+08,523057,10912.405490,2.842262e+08,15.08,16.972956,0.300290,50,"34,560.04","4,293.69",13705.13,80.10
2065,2023-08-29,26120.00,28142.85,25922.00,27716.34,74251.994880,2.023326e+09,1137083,37945.550710,1.034244e+09,14.45,21.151151,0.376069,64,"34,852.87","4,326.47",13943.76,81.16
2066,2023-08-30,27716.34,27768.57,27017.24,27299.99,35448.258480,9.690865e+08,690292,16715.577120,4.569264e+08,13.88,19.220988,0.378955,55,"34,890.77","4,315.31",14019.31,81.63


In [740]:
# mining_difficulty.csv 파일 읽기
spx = pd.read_excel('dataset/SPXdata.xlsx')

In [741]:
import pandas as pd

# spx 데이터의 날짜 형식 통일
spx['Date'] = pd.to_datetime(spx['Date']).dt.date

# spx 데이터에 누락된 날짜 채우기
all_dates = pd.date_range(start=min(spx['Date']), end=max(spx['Date'])).date
spx_full = pd.DataFrame({'Date': all_dates}).merge(spx, on='Date', how='left')

# 누락된 spx 값에 대해 나중의 값으로 대체
spx_full['Close/Last'].fillna(method='bfill', inplace=True)

# 기존의 병합된 데이터프레임과 날짜(date) 기준으로 병합
spxadded = pd.merge(oiladded, spx_full[['Date', 'Close/Last']], left_on='Open_time', right_on='Date', how='inner')

# 중복된 'Date' 열 제거
spxadded.drop('Date', axis=1, inplace=True)

# 'Close/Last' 열의 이름을 'SPX'로 변경
spxadded.rename(columns={'Close/Last': 'SPX'}, inplace=True)

# 변경된 데이터프레임 확인
spxadded


,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,Nasdaq,OilWTI,SPX
0,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,9.77,649.164976,1.616095,53,"24,824.01","3,490.19",7006.90,60.37,2695.81
1,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,9.15,749.685473,1.613762,55,"24,922.68","3,509.88",7065.53,61.63,2713.06
2,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,9.22,778.261095,1.613778,54,"25,075.13","3,568.88",7077.91,62.01,2723.99
3,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,9.22,701.356021,1.580870,52,"25,295.87","3,607.63",7136.56,61.44,2743.15
4,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,3.092169e+08,158242,11007.164056,1.834178e+08,9.52,694.477906,1.378919,49,"25,283.00","3,616.45",7157.39,61.73,2747.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,2023-08-27,26017.38,26182.23,25966.11,26101.77,12099.642160,3.155484e+08,349090,6170.486640,1.609182e+08,15.08,13.471949,0.300414,45,"34,560.04","4,293.69",13705.13,80.10,4433.31
2064,2023-08-28,26101.78,26253.99,25864.50,26120.00,22692.626550,5.910089e+08,523057,10912.405490,2.842262e+08,15.08,16.972956,0.300290,50,"34,560.04","4,293.69",13705.13,80.10,4433.31
2065,2023-08-29,26120.00,28142.85,25922.00,27716.34,74251.994880,2.023326e+09,1137083,37945.550710,1.034244e+09,14.45,21.151151,0.376069,64,"34,852.87","4,326.47",13943.76,81.16,4497.63
2066,2023-08-30,27716.34,27768.57,27017.24,27299.99,35448.258480,9.690865e+08,690292,16715.577120,4.569264e+08,13.88,19.220988,0.378955,55,"34,890.77","4,315.31",14019.31,81.63,4514.87


In [742]:
tpm = pd.read_csv('dataset/trade_per_minute.csv')

---
 - Based on Bitfinex
---

In [743]:
# 날짜 형식 통일
tpm['Time'] = pd.to_datetime(tpm['Time']).dt.date

# 기존의 병합된 데이터프레임과 날짜(date) 기준으로 병합
tpmadded = pd.merge(spxadded, tpm, left_on='Open_time', right_on='Time', how='inner')

# 중복된 'Time' 열 제거
tpmadded.drop('Time', axis=1, inplace=True)

# 최종 데이터프레임 확인
tpmadded

,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,...,bitfinex,bitflyer,bitstamp,btcchina,coinbase,gemini,huobi,kraken,okcoin,others
0,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,...,135.521528,48.967361,64.484722,NaN,193.399306,22.059028,NaN,34.820139,NaN,257.365278
1,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,...,112.597917,47.106250,67.563889,NaN,168.494444,20.569444,NaN,34.290972,NaN,226.798611
2,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,...,124.011806,59.004167,86.456944,NaN,166.814583,26.843750,NaN,42.932639,NaN,248.669444
3,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,...,130.421528,76.375694,81.521528,NaN,188.475000,25.138889,NaN,46.116667,NaN,340.470833
4,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,3.092169e+08,158242,11007.164056,1.834178e+08,...,88.506250,39.659028,49.371528,NaN,158.761806,12.834722,NaN,33.613889,NaN,233.252778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,2023-08-27,26017.38,26182.23,25966.11,26101.77,12099.642160,3.155484e+08,349090,6170.486640,1.609182e+08,...,1.552083,NaN,7.139583,NaN,85.503472,1.836111,NaN,17.351389,NaN,6.806250
2064,2023-08-28,26101.78,26253.99,25864.50,26120.00,22692.626550,5.910089e+08,523057,10912.405490,2.842262e+08,...,5.611806,NaN,13.472917,NaN,135.866667,4.132639,NaN,30.890972,NaN,9.520833
2065,2023-08-29,26120.00,28142.85,25922.00,27716.34,74251.994880,2.023326e+09,1137083,37945.550710,1.034244e+09,...,17.523611,NaN,24.475000,NaN,283.150000,7.945139,NaN,49.247917,NaN,12.987500
2066,2023-08-30,27716.34,27768.57,27017.24,27299.99,35448.258480,9.690865e+08,690292,16715.577120,4.569264e+08,...,8.276389,NaN,16.097917,NaN,182.864583,4.456944,NaN,32.259722,NaN,9.080556


In [744]:
# 'Unnamed: 0', 'price', 'rate' 열 제거
tpmadded.drop(['bitflyer', 'bitstamp', 'btcchina', 'coinbase', 'gemini', 'huobi', 'kraken', 'okcoin','others'], axis=1, inplace=True)

# 'vol' 열의 이름을 'volatility'로 변경
tpmadded.rename(columns={'bitfinex': 'TradePermin'}, inplace=True)

# 변경된 데이터프레임 확인
tpmadded

,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,Nasdaq,OilWTI,SPX,TradePermin
0,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,9.77,649.164976,1.616095,53,"24,824.01","3,490.19",7006.90,60.37,2695.81,135.521528
1,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,9.15,749.685473,1.613762,55,"24,922.68","3,509.88",7065.53,61.63,2713.06,112.597917
2,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,9.22,778.261095,1.613778,54,"25,075.13","3,568.88",7077.91,62.01,2723.99,124.011806
3,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,9.22,701.356021,1.580870,52,"25,295.87","3,607.63",7136.56,61.44,2743.15,130.421528
4,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,3.092169e+08,158242,11007.164056,1.834178e+08,9.52,694.477906,1.378919,49,"25,283.00","3,616.45",7157.39,61.73,2747.71,88.506250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,2023-08-27,26017.38,26182.23,25966.11,26101.77,12099.642160,3.155484e+08,349090,6170.486640,1.609182e+08,15.08,13.471949,0.300414,45,"34,560.04","4,293.69",13705.13,80.10,4433.31,1.552083
2064,2023-08-28,26101.78,26253.99,25864.50,26120.00,22692.626550,5.910089e+08,523057,10912.405490,2.842262e+08,15.08,16.972956,0.300290,50,"34,560.04","4,293.69",13705.13,80.10,4433.31,5.611806
2065,2023-08-29,26120.00,28142.85,25922.00,27716.34,74251.994880,2.023326e+09,1137083,37945.550710,1.034244e+09,14.45,21.151151,0.376069,64,"34,852.87","4,326.47",13943.76,81.16,4497.63,17.523611
2066,2023-08-30,27716.34,27768.57,27017.24,27299.99,35448.258480,9.690865e+08,690292,16715.577120,4.569264e+08,13.88,19.220988,0.378955,55,"34,890.77","4,315.31",14019.31,81.63,4514.87,8.276389


In [745]:
bd = pd.read_csv('dataset/bitcoin_dominance_2017to20230924.csv')

# Dominance column만 남기기
bd = bd[['DateTime','Dominance']]

# 날짜 형식 통일
bd['DateTime'] = pd.to_datetime(bd['DateTime']).dt.date

bd

,DateTime,Dominance
0,2017-01-08,0.857557
1,2017-01-15,0.851250
2,2017-01-22,0.852142
3,2017-01-29,0.851960
4,2017-02-05,0.854416
...,...,...
346,2023-08-27,0.482729
347,2023-09-03,0.482892
348,2023-09-10,0.483074
349,2023-09-17,0.488896


In [746]:
import pandas as pd

# 데이터 불러오기
bd = pd.read_csv('dataset/bitcoin_dominance_2017to20230924.csv')
bd = bd[['DateTime','Dominance']]

# 'DateTime' 컬럼을 datetime 타입으로 변환
bd['DateTime'] = pd.to_datetime(bd['DateTime'])

# 인덱스를 'DateTime'으로 설정
bd.set_index('DateTime', inplace=True)

# 하루 단위로 리샘플링하고 누락된 값을 가장 최근의 값으로 채움
bd_daily = bd.resample('D').ffill()

print(bd_daily)

            Dominance
DateTime             
2017-01-08   0.857557
2017-01-09   0.857557
2017-01-10   0.857557
2017-01-11   0.857557
2017-01-12   0.857557
...               ...
2023-09-20   0.488896
2023-09-21   0.488896
2023-09-22   0.488896
2023-09-23   0.488896
2023-09-24   0.491337

[2451 rows x 1 columns]


In [747]:
# 만약 'Open_time'이 object 타입이라면 datetime으로 변환
if tpmadded['Open_time'].dtype == 'object':
    tpmadded['Open_time'] = pd.to_datetime(tpmadded['Open_time'])

# 병합을 시도하기 전에 'bd_daily'의 인덱스를 열로 리셋
bd_daily_reset = bd_daily.reset_index()

# 이제 병합을 시도
bdadded = pd.merge(tpmadded, bd_daily_reset, left_on='Open_time', right_on='DateTime', how='inner')

# 중복된 'DateTime' 열 제거
bdadded.drop('DateTime', axis=1, inplace=True)

# 최종 데이터프레임 확인
bdadded.head()


,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,...,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,Nasdaq,OilWTI,SPX,TradePermin,Dominance
0,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,...,649.164976,1.616095,53,"24,824.01","3,490.19",7006.90,60.37,2695.81,135.521528,0.378384
1,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,...,749.685473,1.613762,55,"24,922.68","3,509.88",7065.53,61.63,2713.06,112.597917,0.378384
2,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,...,778.261095,1.613778,54,"25,075.13","3,568.88",7077.91,62.01,2723.99,124.011806,0.378384
3,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,...,701.356021,1.580870,52,"25,295.87","3,607.63",7136.56,61.44,2743.15,130.421528,0.378384
4,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,3.092169e+08,158242,11007.164056,1.834178e+08,...,694.477906,1.378919,49,"25,283.00","3,616.45",7157.39,61.73,2747.71,88.506250,0.378384


In [748]:
bdadded

,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,...,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,Nasdaq,OilWTI,SPX,TradePermin,Dominance
0,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,...,649.164976,1.616095,53,"24,824.01","3,490.19",7006.90,60.37,2695.81,135.521528,0.378384
1,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,...,749.685473,1.613762,55,"24,922.68","3,509.88",7065.53,61.63,2713.06,112.597917,0.378384
2,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,...,778.261095,1.613778,54,"25,075.13","3,568.88",7077.91,62.01,2723.99,124.011806,0.378384
3,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,...,701.356021,1.580870,52,"25,295.87","3,607.63",7136.56,61.44,2743.15,130.421528,0.378384
4,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,3.092169e+08,158242,11007.164056,1.834178e+08,...,694.477906,1.378919,49,"25,283.00","3,616.45",7157.39,61.73,2747.71,88.506250,0.378384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,2023-08-27,26017.38,26182.23,25966.11,26101.77,12099.642160,3.155484e+08,349090,6170.486640,1.609182e+08,...,13.471949,0.300414,45,"34,560.04","4,293.69",13705.13,80.10,4433.31,1.552083,0.482729
2064,2023-08-28,26101.78,26253.99,25864.50,26120.00,22692.626550,5.910089e+08,523057,10912.405490,2.842262e+08,...,16.972956,0.300290,50,"34,560.04","4,293.69",13705.13,80.10,4433.31,5.611806,0.482729
2065,2023-08-29,26120.00,28142.85,25922.00,27716.34,74251.994880,2.023326e+09,1137083,37945.550710,1.034244e+09,...,21.151151,0.376069,64,"34,852.87","4,326.47",13943.76,81.16,4497.63,17.523611,0.482729
2066,2023-08-30,27716.34,27768.57,27017.24,27299.99,35448.258480,9.690865e+08,690292,16715.577120,4.569264e+08,...,19.220988,0.378955,55,"34,890.77","4,315.31",14019.31,81.63,4514.87,8.276389,0.482729


In [749]:
gold = pd.read_excel('dataset/gold price.xlsx')
# Dominance column만 남기기
gold = gold[['Unnamed: 0', 'USD']]
# 'vol' 열의 이름을 'volatility'로 변경
gold.rename(columns={'Unnamed: 0': 'Date' , 'USD' : 'Gold'}, inplace=True)
gold

,Date,Gold
0,2018-01-01,1291.00
1,2018-01-02,1312.05
2,2018-01-03,1314.90
3,2018-01-04,1314.50
4,2018-01-05,1317.15
...,...,...
1480,2023-09-04,1937.20
1481,2023-09-05,1926.10
1482,2023-09-06,1922.05
1483,2023-09-07,1918.35


In [750]:
# 'DateTime' 컬럼을 datetime 타입으로 변환
gold['Date'] = pd.to_datetime(gold['Date'])

# 인덱스를 'DateTime'으로 설정
gold.set_index('Date', inplace=True)

# 하루 단위로 리샘플링하고 누락된 값을 가장 최근의 값으로 채움
gold_daily = gold.resample('D').ffill()

gold_daily

,Gold
Date,
2018-01-01,1291.00
2018-01-02,1312.05
2018-01-03,1314.90
2018-01-04,1314.50
2018-01-05,1317.15
...,...
2023-09-04,1937.20
2023-09-05,1926.10
2023-09-06,1922.05


In [751]:
# 만약 'Open_time'이 object 타입이라면 datetime으로 변환
if bdadded['Open_time'].dtype == 'object':
    bdadded['Open_time'] = pd.to_datetime(bdadded['Open_time'])

# 병합을 시도하기 전에 'bd_daily'의 인덱스를 열로 리셋
gold_daily_reset = gold_daily.reset_index()

# 이제 병합을 시도
goldadded = pd.merge(bdadded, gold_daily_reset, left_on='Open_time', right_on='Date', how='inner')

# 중복된 'Date' 열 제거
goldadded.drop('Date', axis=1, inplace=True)

# 최종 데이터프레임 확인
goldadded.head()

,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,...,volatility,GoogleTrend,DowJones,EuroStoxx50,Nasdaq,OilWTI,SPX,TradePermin,Dominance,Gold
0,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,...,1.616095,53,"24,824.01","3,490.19",7006.90,60.37,2695.81,135.521528,0.378384,1312.05
1,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,...,1.613762,55,"24,922.68","3,509.88",7065.53,61.63,2713.06,112.597917,0.378384,1314.90
2,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,...,1.613778,54,"25,075.13","3,568.88",7077.91,62.01,2723.99,124.011806,0.378384,1314.50
3,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,...,1.580870,52,"25,295.87","3,607.63",7136.56,61.44,2743.15,130.421528,0.378384,1317.15
4,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,3.092169e+08,158242,11007.164056,1.834178e+08,...,1.378919,49,"25,283.00","3,616.45",7157.39,61.73,2747.71,88.506250,0.378384,1317.15


In [752]:
goldadded

,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,...,volatility,GoogleTrend,DowJones,EuroStoxx50,Nasdaq,OilWTI,SPX,TradePermin,Dominance,Gold
0,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,...,1.616095,53,"24,824.01","3,490.19",7006.90,60.37,2695.81,135.521528,0.378384,1312.05
1,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,...,1.613762,55,"24,922.68","3,509.88",7065.53,61.63,2713.06,112.597917,0.378384,1314.90
2,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,...,1.613778,54,"25,075.13","3,568.88",7077.91,62.01,2723.99,124.011806,0.378384,1314.50
3,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,...,1.580870,52,"25,295.87","3,607.63",7136.56,61.44,2743.15,130.421528,0.378384,1317.15
4,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,3.092169e+08,158242,11007.164056,1.834178e+08,...,1.378919,49,"25,283.00","3,616.45",7157.39,61.73,2747.71,88.506250,0.378384,1317.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,2023-08-27,26017.38,26182.23,25966.11,26101.77,12099.642160,3.155484e+08,349090,6170.486640,1.609182e+08,...,0.300414,45,"34,560.04","4,293.69",13705.13,80.10,4433.31,1.552083,0.482729,1915.50
2064,2023-08-28,26101.78,26253.99,25864.50,26120.00,22692.626550,5.910089e+08,523057,10912.405490,2.842262e+08,...,0.300290,50,"34,560.04","4,293.69",13705.13,80.10,4433.31,5.611806,0.482729,1915.50
2065,2023-08-29,26120.00,28142.85,25922.00,27716.34,74251.994880,2.023326e+09,1137083,37945.550710,1.034244e+09,...,0.376069,64,"34,852.87","4,326.47",13943.76,81.16,4497.63,17.523611,0.482729,1930.00
2066,2023-08-30,27716.34,27768.57,27017.24,27299.99,35448.258480,9.690865e+08,690292,16715.577120,4.569264e+08,...,0.378955,55,"34,890.77","4,315.31",14019.31,81.63,4514.87,8.276389,0.482729,1947.55


In [753]:
# 데이터 불러오기
mchi = pd.read_csv('dataset/MCHI.csv')
mchi = mchi[['Date','Close']]

# 'DateTime' 컬럼을 datetime 타입으로 변환
mchi['Date'] = pd.to_datetime(mchi['Date'])

# 인덱스를 'DateTime'으로 설정
mchi.set_index('Date', inplace=True)

# 하루 단위로 리샘플링하고 누락된 값을 가장 최근의 값으로 채움
mchi_daily = mchi.resample('D').ffill()

print(mchi_daily)

                Close
Date                 
2011-03-31  51.900002
2011-04-01  52.619999
2011-04-02  52.619999
2011-04-03  52.619999
2011-04-04  53.740002
...               ...
2023-10-30  42.459999
2023-10-31  41.810001
2023-11-01  41.799999
2023-11-02  42.150002
2023-11-03  43.250000

[4601 rows x 1 columns]


In [754]:
# 만약 'Open_time'이 object 타입이라면 datetime으로 변환
if goldadded['Open_time'].dtype == 'object':
    goldadded['Open_time'] = pd.to_datetime(goldadded['Open_time'])

# 병합을 시도하기 전에 'bd_daily'의 인덱스를 열로 리셋
mchi_daily_reset = mchi_daily.reset_index()

# 이제 병합을 시도
mchiadded = pd.merge(goldadded, mchi_daily_reset, left_on='Open_time', right_on='Date', how='inner')

# 중복된 'Date' 열 제거
mchiadded.drop('Date', axis=1, inplace=True)

# 최종 데이터프레임 확인
mchiadded.head()

,Open_time,Open,High,Low,Close_x,Volume,quote_av,trades,tb_base_av,tb_quote_av,...,GoogleTrend,DowJones,EuroStoxx50,Nasdaq,OilWTI,SPX,TradePermin,Dominance,Gold,Close_y
0,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,...,53,"24,824.01","3,490.19",7006.90,60.37,2695.81,135.521528,0.378384,1312.05,69.129997
1,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,...,55,"24,922.68","3,509.88",7065.53,61.63,2713.06,112.597917,0.378384,1314.90,69.680000
2,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,...,54,"25,075.13","3,568.88",7077.91,62.01,2723.99,124.011806,0.378384,1314.50,70.110001
3,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,...,52,"25,295.87","3,607.63",7136.56,61.44,2743.15,130.421528,0.378384,1317.15,70.610001
4,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,3.092169e+08,158242,11007.164056,1.834178e+08,...,49,"25,283.00","3,616.45",7157.39,61.73,2747.71,88.506250,0.378384,1317.15,70.610001


In [755]:
mchiadded.rename(columns={'Close_x' : 'Close' , 'Close_y' : 'MCHI'}, inplace=True)

mchiadded

,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,...,GoogleTrend,DowJones,EuroStoxx50,Nasdaq,OilWTI,SPX,TradePermin,Dominance,Gold,MCHI
0,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,...,53,"24,824.01","3,490.19",7006.90,60.37,2695.81,135.521528,0.378384,1312.05,69.129997
1,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,...,55,"24,922.68","3,509.88",7065.53,61.63,2713.06,112.597917,0.378384,1314.90,69.680000
2,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,...,54,"25,075.13","3,568.88",7077.91,62.01,2723.99,124.011806,0.378384,1314.50,70.110001
3,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,...,52,"25,295.87","3,607.63",7136.56,61.44,2743.15,130.421528,0.378384,1317.15,70.610001
4,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,3.092169e+08,158242,11007.164056,1.834178e+08,...,49,"25,283.00","3,616.45",7157.39,61.73,2747.71,88.506250,0.378384,1317.15,70.610001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,2023-08-27,26017.38,26182.23,25966.11,26101.77,12099.642160,3.155484e+08,349090,6170.486640,1.609182e+08,...,45,"34,560.04","4,293.69",13705.13,80.10,4433.31,1.552083,0.482729,1915.50,43.759998
2064,2023-08-28,26101.78,26253.99,25864.50,26120.00,22692.626550,5.910089e+08,523057,10912.405490,2.842262e+08,...,50,"34,560.04","4,293.69",13705.13,80.10,4433.31,5.611806,0.482729,1915.50,44.650002
2065,2023-08-29,26120.00,28142.85,25922.00,27716.34,74251.994880,2.023326e+09,1137083,37945.550710,1.034244e+09,...,64,"34,852.87","4,326.47",13943.76,81.16,4497.63,17.523611,0.482729,1930.00,45.619999
2066,2023-08-30,27716.34,27768.57,27017.24,27299.99,35448.258480,9.690865e+08,690292,16715.577120,4.569264e+08,...,55,"34,890.77","4,315.31",14019.31,81.63,4514.87,8.276389,0.482729,1947.55,45.369999


In [756]:
mchiadded.to_csv('Dataset/CombinedFeature.csv', index=False)

최종 22개의 FEATURES

In [757]:
missing_values = mchiadded.isna().sum()
print(missing_values)

Open_time                 0
Open                      0
High                      0
Low                       0
Close                     0
Volume                    0
quote_av                  0
trades                    0
tb_base_av                0
tb_quote_av               0
VIX                       0
Bitcoin_TransactionFee    0
volatility                0
GoogleTrend               0
DowJones                  0
EuroStoxx50               0
Nasdaq                    0
OilWTI                    0
SPX                       0
TradePermin               0
Dominance                 0
Gold                      0
MCHI                      0
dtype: int64


In [758]:
# 쉼표 제거 후 float 타입으로 변환
mchiadded['DowJones'] = mchiadded['DowJones'].str.replace(',', '').astype(float)


# 변환 후의 데이터프레임 확인
mchiadded

,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,...,GoogleTrend,DowJones,EuroStoxx50,Nasdaq,OilWTI,SPX,TradePermin,Dominance,Gold,MCHI
0,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,...,53,24824.01,"3,490.19",7006.90,60.37,2695.81,135.521528,0.378384,1312.05,69.129997
1,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,...,55,24922.68,"3,509.88",7065.53,61.63,2713.06,112.597917,0.378384,1314.90,69.680000
2,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,...,54,25075.13,"3,568.88",7077.91,62.01,2723.99,124.011806,0.378384,1314.50,70.110001
3,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,...,52,25295.87,"3,607.63",7136.56,61.44,2743.15,130.421528,0.378384,1317.15,70.610001
4,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,3.092169e+08,158242,11007.164056,1.834178e+08,...,49,25283.00,"3,616.45",7157.39,61.73,2747.71,88.506250,0.378384,1317.15,70.610001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,2023-08-27,26017.38,26182.23,25966.11,26101.77,12099.642160,3.155484e+08,349090,6170.486640,1.609182e+08,...,45,34560.04,"4,293.69",13705.13,80.10,4433.31,1.552083,0.482729,1915.50,43.759998
2064,2023-08-28,26101.78,26253.99,25864.50,26120.00,22692.626550,5.910089e+08,523057,10912.405490,2.842262e+08,...,50,34560.04,"4,293.69",13705.13,80.10,4433.31,5.611806,0.482729,1915.50,44.650002
2065,2023-08-29,26120.00,28142.85,25922.00,27716.34,74251.994880,2.023326e+09,1137083,37945.550710,1.034244e+09,...,64,34852.87,"4,326.47",13943.76,81.16,4497.63,17.523611,0.482729,1930.00,45.619999
2066,2023-08-30,27716.34,27768.57,27017.24,27299.99,35448.258480,9.690865e+08,690292,16715.577120,4.569264e+08,...,55,34890.77,"4,315.31",14019.31,81.63,4514.87,8.276389,0.482729,1947.55,45.369999


In [759]:
# 쉼표 제거 후 float 타입으로 변환
mchiadded['EuroStoxx50'] = mchiadded['EuroStoxx50'].str.replace(',', '').astype(float)


# 변환 후의 데이터프레임 확인
goldadded

,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,...,volatility,GoogleTrend,DowJones,EuroStoxx50,Nasdaq,OilWTI,SPX,TradePermin,Dominance,Gold
0,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,...,1.616095,53,"24,824.01","3,490.19",7006.90,60.37,2695.81,135.521528,0.378384,1312.05
1,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,...,1.613762,55,"24,922.68","3,509.88",7065.53,61.63,2713.06,112.597917,0.378384,1314.90
2,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,...,1.613778,54,"25,075.13","3,568.88",7077.91,62.01,2723.99,124.011806,0.378384,1314.50
3,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,...,1.580870,52,"25,295.87","3,607.63",7136.56,61.44,2743.15,130.421528,0.378384,1317.15
4,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,3.092169e+08,158242,11007.164056,1.834178e+08,...,1.378919,49,"25,283.00","3,616.45",7157.39,61.73,2747.71,88.506250,0.378384,1317.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,2023-08-27,26017.38,26182.23,25966.11,26101.77,12099.642160,3.155484e+08,349090,6170.486640,1.609182e+08,...,0.300414,45,"34,560.04","4,293.69",13705.13,80.10,4433.31,1.552083,0.482729,1915.50
2064,2023-08-28,26101.78,26253.99,25864.50,26120.00,22692.626550,5.910089e+08,523057,10912.405490,2.842262e+08,...,0.300290,50,"34,560.04","4,293.69",13705.13,80.10,4433.31,5.611806,0.482729,1915.50
2065,2023-08-29,26120.00,28142.85,25922.00,27716.34,74251.994880,2.023326e+09,1137083,37945.550710,1.034244e+09,...,0.376069,64,"34,852.87","4,326.47",13943.76,81.16,4497.63,17.523611,0.482729,1930.00
2066,2023-08-30,27716.34,27768.57,27017.24,27299.99,35448.258480,9.690865e+08,690292,16715.577120,4.569264e+08,...,0.378955,55,"34,890.77","4,315.31",14019.31,81.63,4514.87,8.276389,0.482729,1947.55


In [760]:
# 변환 후의 데이터프레임 확인
mchiadded

,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,...,GoogleTrend,DowJones,EuroStoxx50,Nasdaq,OilWTI,SPX,TradePermin,Dominance,Gold,MCHI
0,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2.797171e+08,177728,11346.326739,1.580801e+08,...,53,24824.01,3490.19,7006.90,60.37,2695.81,135.521528,0.378384,1312.05,69.129997
1,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2.361169e+08,162787,8994.953566,1.335873e+08,...,55,24922.68,3509.88,7065.53,61.63,2713.06,112.597917,0.378384,1314.90,69.680000
2,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,3.127816e+08,170310,12680.812951,1.861168e+08,...,54,25075.13,3568.88,7077.91,62.01,2723.99,124.011806,0.378384,1314.50,70.110001
3,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,3.693220e+08,192969,13346.622293,2.118299e+08,...,52,25295.87,3607.63,7136.56,61.44,2743.15,130.421528,0.378384,1317.15,70.610001
4,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,3.092169e+08,158242,11007.164056,1.834178e+08,...,49,25283.00,3616.45,7157.39,61.73,2747.71,88.506250,0.378384,1317.15,70.610001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,2023-08-27,26017.38,26182.23,25966.11,26101.77,12099.642160,3.155484e+08,349090,6170.486640,1.609182e+08,...,45,34560.04,4293.69,13705.13,80.10,4433.31,1.552083,0.482729,1915.50,43.759998
2064,2023-08-28,26101.78,26253.99,25864.50,26120.00,22692.626550,5.910089e+08,523057,10912.405490,2.842262e+08,...,50,34560.04,4293.69,13705.13,80.10,4433.31,5.611806,0.482729,1915.50,44.650002
2065,2023-08-29,26120.00,28142.85,25922.00,27716.34,74251.994880,2.023326e+09,1137083,37945.550710,1.034244e+09,...,64,34852.87,4326.47,13943.76,81.16,4497.63,17.523611,0.482729,1930.00,45.619999
2066,2023-08-30,27716.34,27768.57,27017.24,27299.99,35448.258480,9.690865e+08,690292,16715.577120,4.569264e+08,...,55,34890.77,4315.31,14019.31,81.63,4514.87,8.276389,0.482729,1947.55,45.369999


In [761]:
mchiadded = mchiadded.drop(columns=[
    'quote_av',
    'tb_base_av',
    'tb_quote_av',
    'Open',
    'High',
    'Low'
])

mchiadded

,Open_time,Close,Volume,trades,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,Nasdaq,OilWTI,SPX,TradePermin,Dominance,Gold,MCHI
0,2018-01-02,14675.11,20078.092111,177728,9.77,649.164976,1.616095,53,24824.01,3490.19,7006.90,60.37,2695.81,135.521528,0.378384,1312.05,69.129997
1,2018-01-03,14919.51,15905.667639,162787,9.15,749.685473,1.613762,55,24922.68,3509.88,7065.53,61.63,2713.06,112.597917,0.378384,1314.90,69.680000
2,2018-01-04,15059.54,21329.649574,170310,9.22,778.261095,1.613778,54,25075.13,3568.88,7077.91,62.01,2723.99,124.011806,0.378384,1314.50,70.110001
3,2018-01-05,16960.39,23251.491125,192969,9.22,701.356021,1.580870,52,25295.87,3607.63,7136.56,61.44,2743.15,130.421528,0.378384,1317.15,70.610001
4,2018-01-06,17069.79,18571.457508,158242,9.52,694.477906,1.378919,49,25283.00,3616.45,7157.39,61.73,2747.71,88.506250,0.378384,1317.15,70.610001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,2023-08-27,26101.77,12099.642160,349090,15.08,13.471949,0.300414,45,34560.04,4293.69,13705.13,80.10,4433.31,1.552083,0.482729,1915.50,43.759998
2064,2023-08-28,26120.00,22692.626550,523057,15.08,16.972956,0.300290,50,34560.04,4293.69,13705.13,80.10,4433.31,5.611806,0.482729,1915.50,44.650002
2065,2023-08-29,27716.34,74251.994880,1137083,14.45,21.151151,0.376069,64,34852.87,4326.47,13943.76,81.16,4497.63,17.523611,0.482729,1930.00,45.619999
2066,2023-08-30,27299.99,35448.258480,690292,13.88,19.220988,0.378955,55,34890.77,4315.31,14019.31,81.63,4514.87,8.276389,0.482729,1947.55,45.369999


In [762]:
# Calculate weekly return
mchiadded['Weekly_return'] = mchiadded['Close'].pct_change(periods=7)

# Calculate monthly return
mchiadded['Monthly_return'] = mchiadded['Close'].pct_change(periods=30)

# Calculate volatility (standard deviation of daily returns)
mchiadded['Daily_return'] = mchiadded['Close'].pct_change()
mchiadded['Volatility'] = mchiadded['Daily_return'].rolling(window=30).std()

# Display the dataframe with the new columns
mchiadded


,Open_time,Close,Volume,trades,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,...,OilWTI,SPX,TradePermin,Dominance,Gold,MCHI,Weekly_return,Monthly_return,Daily_return,Volatility
0,2018-01-02,14675.11,20078.092111,177728,9.77,649.164976,1.616095,53,24824.01,3490.19,...,60.37,2695.81,135.521528,0.378384,1312.05,69.129997,NaN,NaN,NaN,NaN
1,2018-01-03,14919.51,15905.667639,162787,9.15,749.685473,1.613762,55,24922.68,3509.88,...,61.63,2713.06,112.597917,0.378384,1314.90,69.680000,NaN,NaN,0.016654,NaN
2,2018-01-04,15059.54,21329.649574,170310,9.22,778.261095,1.613778,54,25075.13,3568.88,...,62.01,2723.99,124.011806,0.378384,1314.50,70.110001,NaN,NaN,0.009386,NaN
3,2018-01-05,16960.39,23251.491125,192969,9.22,701.356021,1.580870,52,25295.87,3607.63,...,61.44,2743.15,130.421528,0.378384,1317.15,70.610001,NaN,NaN,0.126222,NaN
4,2018-01-06,17069.79,18571.457508,158242,9.52,694.477906,1.378919,49,25283.00,3616.45,...,61.73,2747.71,88.506250,0.378384,1317.15,70.610001,NaN,NaN,0.006450,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,2023-08-27,26101.77,12099.642160,349090,15.08,13.471949,0.300414,45,34560.04,4293.69,...,80.10,4433.31,1.552083,0.482729,1915.50,43.759998,-0.003368,-0.109584,0.003244,0.015724
2064,2023-08-28,26120.00,22692.626550,523057,15.08,16.972956,0.300290,50,34560.04,4293.69,...,80.10,4433.31,5.611806,0.482729,1915.50,44.650002,-0.000265,-0.110139,0.000698,0.015718
2065,2023-08-29,27716.34,74251.994880,1137083,14.45,21.151151,0.376069,64,34852.87,4326.47,...,81.16,4497.63,17.523611,0.482729,1930.00,45.619999,0.063722,-0.053439,0.061116,0.019684
2066,2023-08-30,27299.99,35448.258480,690292,13.88,19.220988,0.378955,55,34890.77,4315.31,...,81.63,4514.87,8.276389,0.482729,1947.55,45.369999,0.032810,-0.066100,-0.015022,0.019835


In [763]:
def calculate_sma(data, window):
    """Calculate the Simple Moving Average (SMA)."""
    return data.rolling(window=window).mean()

def calculate_ema(data, window):
    """Calculate the Exponential Moving Average (EMA)."""
    return data.ewm(span=window, adjust=False).mean()

def calculate_rsi(data, window):
    """Calculate the Relative Strength Index (RSI)."""
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    return 100 - (100 / (1 + rs))

In [764]:
# Apply technical analysis indicators to the 'Close' column using pandas_ta
# Example indicators: SMA (Simple Moving Average), EMA (Exponential Moving Average), RSI (Relative Strength Index)
mchiadded['SMA_10'] = ta.sma(mchiadded['Close'], length=10)
mchiadded['EMA_10'] = ta.ema(mchiadded['Close'], length=10)
mchiadded['RSI_14'] = ta.rsi(mchiadded['Close'], length=14)


In [765]:
def calculate_macd(data, short_window, long_window, signal_window):
    """Calculate the Moving Average Convergence Divergence (MACD)."""
    short_ema = calculate_ema(data, short_window)
    long_ema = calculate_ema(data, long_window)
    macd = short_ema - long_ema
    signal_line = calculate_ema(macd, signal_window)
    return macd, signal_line

def calculate_bollinger_bands(data, window, no_of_std):
    """Calculate Bollinger Bands."""
    sma = calculate_sma(data, window)
    rstd = data.rolling(window=window).std()
    upper_band = sma + rstd * no_of_std
    lower_band = sma - rstd * no_of_std
    return upper_band, lower_band

# Apply MACD
mchiadded['MACD'], mchiadded['MACD_Signal'] = calculate_macd(mchiadded['Close'], 12, 26, 9)

# Apply Bollinger Bands
mchiadded['Bollinger_Upper'], mchiadded['Bollinger_Lower'] = calculate_bollinger_bands(mchiadded['Close'], 20, 2)

mchiadded

,Open_time,Close,Volume,trades,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,...,Monthly_return,Daily_return,Volatility,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,Bollinger_Upper,Bollinger_Lower
0,2018-01-02,14675.11,20078.092111,177728,9.77,649.164976,1.616095,53,24824.01,3490.19,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN
1,2018-01-03,14919.51,15905.667639,162787,9.15,749.685473,1.613762,55,24922.68,3509.88,...,NaN,0.016654,NaN,NaN,NaN,NaN,19.496296,3.899259,NaN,NaN
2,2018-01-04,15059.54,21329.649574,170310,9.22,778.261095,1.613778,54,25075.13,3568.88,...,NaN,0.009386,NaN,NaN,NaN,NaN,45.719477,12.263303,NaN,NaN
3,2018-01-05,16960.39,23251.491125,192969,9.22,701.356021,1.580870,52,25295.87,3607.63,...,NaN,0.126222,NaN,NaN,NaN,NaN,217.378442,53.286330,NaN,NaN
4,2018-01-06,17069.79,18571.457508,158242,9.52,694.477906,1.378919,49,25283.00,3616.45,...,NaN,0.006450,NaN,NaN,NaN,NaN,358.118902,114.252845,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,2023-08-27,26101.77,12099.642160,349090,15.08,13.471949,0.300414,45,34560.04,4293.69,...,-0.109584,0.003244,0.015724,26131.884,26478.236351,26.749939,-904.471454,-800.659263,30906.076489,24321.112511
2064,2023-08-28,26120.00,22692.626550,523057,15.08,16.972956,0.300290,50,34560.04,4293.69,...,-0.110139,0.000698,0.015718,26138.484,26413.102469,27.164624,-878.947958,-816.317002,30623.324935,24238.822065
2065,2023-08-29,27716.34,74251.994880,1137083,14.45,21.151151,0.376069,64,34852.87,4326.47,...,-0.053439,0.061116,0.019684,26300.117,26650.054748,52.515235,-721.591055,-797.371813,30370.442488,24305.139512
2066,2023-08-30,27299.99,35448.258480,690292,13.88,19.220988,0.378955,55,34890.77,4315.31,...,-0.066100,-0.015022,0.019835,26411.117,26768.224793,47.838513,-623.295577,-762.556566,30094.263537,24365.742463


In [766]:
mchiadded = mchiadded.iloc[30:].reset_index(drop=True)
mchiadded

,Open_time,Close,Volume,trades,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,...,Monthly_return,Daily_return,Volatility,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,Bollinger_Upper,Bollinger_Lower
0,2018-02-01,9224.52,33564.764311,316020,13.47,214.027545,1.304402,50,26186.71,3577.35,...,-0.371417,-0.103118,0.068276,10878.252,10778.001293,30.766722,-980.636763,-882.651939,13889.562049,9052.754951
1,2018-02-02,8873.03,49971.626975,453411,17.31,274.362582,1.302707,66,25520.96,3523.28,...,-0.405273,-0.038104,0.068187,10685.637,10431.642876,29.325947,-1072.903152,-920.702181,13526.109039,8882.510961
2,2018-02-03,9199.96,28725.000735,327519,37.32,146.927769,1.312619,41,24345.75,3478.77,...,-0.389094,0.036845,0.068706,10470.634,10207.700535,32.492548,-1106.884930,-957.938731,13217.366619,8763.750381
3,2018-02-04,8184.81,32014.308449,356134,37.32,111.316698,1.252705,38,24345.75,3478.77,...,-0.517416,-0.110343,0.065570,10171.588,9839.902256,28.258612,-1201.875384,-1006.726062,12946.728260,8498.876740
4,2018-02-05,6939.99,63403.182579,533510,37.32,173.767502,1.327510,63,24345.75,3478.77,...,-0.593434,-0.152089,0.069485,9756.687,9312.645482,24.109870,-1361.903399,-1077.761529,13315.272814,7734.331186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2033,2023-08-27,26101.77,12099.642160,349090,15.08,13.471949,0.300414,45,34560.04,4293.69,...,-0.109584,0.003244,0.015724,26131.884,26478.236351,26.749939,-904.471454,-800.659263,30906.076489,24321.112511
2034,2023-08-28,26120.00,22692.626550,523057,15.08,16.972956,0.300290,50,34560.04,4293.69,...,-0.110139,0.000698,0.015718,26138.484,26413.102469,27.164624,-878.947958,-816.317002,30623.324935,24238.822065
2035,2023-08-29,27716.34,74251.994880,1137083,14.45,21.151151,0.376069,64,34852.87,4326.47,...,-0.053439,0.061116,0.019684,26300.117,26650.054748,52.515235,-721.591055,-797.371813,30370.442488,24305.139512
2036,2023-08-30,27299.99,35448.258480,690292,13.88,19.220988,0.378955,55,34890.77,4315.31,...,-0.066100,-0.015022,0.019835,26411.117,26768.224793,47.838513,-623.295577,-762.556566,30094.263537,24365.742463


In [767]:
mchiadded.to_csv('Dataset/breakpoint.csv', index=False)

여기서 volatility close 이용해서 구하고 feature 로 넣기

Weekly return, monthly return, pandas_ta 이용해서 Technical Indicator 보여주기

In [768]:
# 'Close' 열의 백분율 변화 계산
mchiadded['Close'] = mchiadded['Close'].pct_change()
mchiadded['DowJones'] = mchiadded['DowJones'].pct_change()
mchiadded['EuroStoxx50'] = mchiadded['EuroStoxx50'].pct_change()
mchiadded['Nasdaq'] = mchiadded['Nasdaq'].pct_change()
mchiadded['OilWTI'] = mchiadded['OilWTI'].pct_change()
mchiadded['SPX'] = mchiadded['SPX'].pct_change()
mchiadded['Gold'] = mchiadded['Gold'].pct_change()
mchiadded['MCHI'] = mchiadded['MCHI'].pct_change()


In [769]:
mchiadded

,Open_time,Close,Volume,trades,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,...,Monthly_return,Daily_return,Volatility,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,Bollinger_Upper,Bollinger_Lower
0,2018-02-01,NaN,33564.764311,316020,13.47,214.027545,1.304402,50,NaN,NaN,...,-0.371417,-0.103118,0.068276,10878.252,10778.001293,30.766722,-980.636763,-882.651939,13889.562049,9052.754951
1,2018-02-02,-0.038104,49971.626975,453411,17.31,274.362582,1.302707,66,-0.025423,-0.015115,...,-0.405273,-0.038104,0.068187,10685.637,10431.642876,29.325947,-1072.903152,-920.702181,13526.109039,8882.510961
2,2018-02-03,0.036845,28725.000735,327519,37.32,146.927769,1.312619,41,-0.046049,-0.012633,...,-0.389094,0.036845,0.068706,10470.634,10207.700535,32.492548,-1106.884930,-957.938731,13217.366619,8763.750381
3,2018-02-04,-0.110343,32014.308449,356134,37.32,111.316698,1.252705,38,0.000000,0.000000,...,-0.517416,-0.110343,0.065570,10171.588,9839.902256,28.258612,-1201.875384,-1006.726062,12946.728260,8498.876740
4,2018-02-05,-0.152089,63403.182579,533510,37.32,173.767502,1.327510,63,0.000000,0.000000,...,-0.593434,-0.152089,0.069485,9756.687,9312.645482,24.109870,-1361.903399,-1077.761529,13315.272814,7734.331186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2033,2023-08-27,0.003244,12099.642160,349090,15.08,13.471949,0.300414,45,0.000000,0.000000,...,-0.109584,0.003244,0.015724,26131.884,26478.236351,26.749939,-904.471454,-800.659263,30906.076489,24321.112511
2034,2023-08-28,0.000698,22692.626550,523057,15.08,16.972956,0.300290,50,0.000000,0.000000,...,-0.110139,0.000698,0.015718,26138.484,26413.102469,27.164624,-878.947958,-816.317002,30623.324935,24238.822065
2035,2023-08-29,0.061116,74251.994880,1137083,14.45,21.151151,0.376069,64,0.008473,0.007634,...,-0.053439,0.061116,0.019684,26300.117,26650.054748,52.515235,-721.591055,-797.371813,30370.442488,24305.139512
2036,2023-08-30,-0.015022,35448.258480,690292,13.88,19.220988,0.378955,55,0.001087,-0.002579,...,-0.066100,-0.015022,0.019835,26411.117,26768.224793,47.838513,-623.295577,-762.556566,30094.263537,24365.742463


In [770]:
# 첫 번째 행 제거
mchiadded = mchiadded.drop(mchiadded.index[0])

# 결과 확인

mchiadded

,Open_time,Close,Volume,trades,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,...,Monthly_return,Daily_return,Volatility,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,Bollinger_Upper,Bollinger_Lower
1,2018-02-02,-0.038104,49971.626975,453411,17.31,274.362582,1.302707,66,-0.025423,-0.015115,...,-0.405273,-0.038104,0.068187,10685.637,10431.642876,29.325947,-1072.903152,-920.702181,13526.109039,8882.510961
2,2018-02-03,0.036845,28725.000735,327519,37.32,146.927769,1.312619,41,-0.046049,-0.012633,...,-0.389094,0.036845,0.068706,10470.634,10207.700535,32.492548,-1106.884930,-957.938731,13217.366619,8763.750381
3,2018-02-04,-0.110343,32014.308449,356134,37.32,111.316698,1.252705,38,0.000000,0.000000,...,-0.517416,-0.110343,0.065570,10171.588,9839.902256,28.258612,-1201.875384,-1006.726062,12946.728260,8498.876740
4,2018-02-05,-0.152089,63403.182579,533510,37.32,173.767502,1.327510,63,0.000000,0.000000,...,-0.593434,-0.152089,0.069485,9756.687,9312.645482,24.109870,-1361.903399,-1077.761529,13315.272814,7734.331186
5,2018-02-06,0.102615,100201.500307,785996,29.98,164.269986,1.398123,79,0.023290,-0.024103,...,-0.526184,0.102615,0.073181,9372.801,9010.735394,30.404847,-1414.951520,-1145.199527,13417.628766,7298.310234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2033,2023-08-27,0.003244,12099.642160,349090,15.08,13.471949,0.300414,45,0.000000,0.000000,...,-0.109584,0.003244,0.015724,26131.884,26478.236351,26.749939,-904.471454,-800.659263,30906.076489,24321.112511
2034,2023-08-28,0.000698,22692.626550,523057,15.08,16.972956,0.300290,50,0.000000,0.000000,...,-0.110139,0.000698,0.015718,26138.484,26413.102469,27.164624,-878.947958,-816.317002,30623.324935,24238.822065
2035,2023-08-29,0.061116,74251.994880,1137083,14.45,21.151151,0.376069,64,0.008473,0.007634,...,-0.053439,0.061116,0.019684,26300.117,26650.054748,52.515235,-721.591055,-797.371813,30370.442488,24305.139512
2036,2023-08-30,-0.015022,35448.258480,690292,13.88,19.220988,0.378955,55,0.001087,-0.002579,...,-0.066100,-0.015022,0.019835,26411.117,26768.224793,47.838513,-623.295577,-762.556566,30094.263537,24365.742463


In [771]:
# Remove 'Close' column and set 'Open_time' as index for the 'mchiadded' dataframe
mchiadded.drop('Daily_return', axis=1, inplace=True)

# The dataframe already has 'Open_time' as its index, so no further action is needed for that
mchiadded

,Open_time,Close,Volume,trades,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,...,Weekly_return,Monthly_return,Volatility,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,Bollinger_Upper,Bollinger_Lower
1,2018-02-02,-0.038104,49971.626975,453411,17.31,274.362582,1.302707,66,-0.025423,-0.015115,...,-0.199835,-0.405273,0.068187,10685.637,10431.642876,29.325947,-1072.903152,-920.702181,13526.109039,8882.510961
2,2018-02-03,0.036845,28725.000735,327519,37.32,146.927769,1.312619,41,-0.046049,-0.012633,...,-0.199377,-0.389094,0.068706,10470.634,10207.700535,32.492548,-1106.884930,-957.938731,13217.366619,8763.750381
3,2018-02-04,-0.110343,32014.308449,356134,37.32,111.316698,1.252705,38,0.000000,0.000000,...,-0.311040,-0.517416,0.065570,10171.588,9839.902256,28.258612,-1201.875384,-1006.726062,12946.728260,8498.876740
4,2018-02-05,-0.152089,63403.182579,533510,37.32,173.767502,1.327510,63,0.000000,0.000000,...,-0.383167,-0.593434,0.069485,9756.687,9312.645482,24.109870,-1361.903399,-1077.761529,13315.272814,7734.331186
5,2018-02-06,0.102615,100201.500307,785996,29.98,164.269986,1.398123,79,0.023290,-0.024103,...,-0.252539,-0.526184,0.073181,9372.801,9010.735394,30.404847,-1414.951520,-1145.199527,13417.628766,7298.310234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2033,2023-08-27,0.003244,12099.642160,349090,15.08,13.471949,0.300414,45,0.000000,0.000000,...,-0.003368,-0.109584,0.015724,26131.884,26478.236351,26.749939,-904.471454,-800.659263,30906.076489,24321.112511
2034,2023-08-28,0.000698,22692.626550,523057,15.08,16.972956,0.300290,50,0.000000,0.000000,...,-0.000265,-0.110139,0.015718,26138.484,26413.102469,27.164624,-878.947958,-816.317002,30623.324935,24238.822065
2035,2023-08-29,0.061116,74251.994880,1137083,14.45,21.151151,0.376069,64,0.008473,0.007634,...,0.063722,-0.053439,0.019684,26300.117,26650.054748,52.515235,-721.591055,-797.371813,30370.442488,24305.139512
2036,2023-08-30,-0.015022,35448.258480,690292,13.88,19.220988,0.378955,55,0.001087,-0.002579,...,0.032810,-0.066100,0.019835,26411.117,26768.224793,47.838513,-623.295577,-762.556566,30094.263537,24365.742463


In [772]:
# 'close' 컬럼을 한 칸씩 위로 올리기
mchiadded['Close'] = mchiadded['Close'].shift(-1)

mchiadded

,Open_time,Close,Volume,trades,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,...,Weekly_return,Monthly_return,Volatility,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,Bollinger_Upper,Bollinger_Lower
1,2018-02-02,0.036845,49971.626975,453411,17.31,274.362582,1.302707,66,-0.025423,-0.015115,...,-0.199835,-0.405273,0.068187,10685.637,10431.642876,29.325947,-1072.903152,-920.702181,13526.109039,8882.510961
2,2018-02-03,-0.110343,28725.000735,327519,37.32,146.927769,1.312619,41,-0.046049,-0.012633,...,-0.199377,-0.389094,0.068706,10470.634,10207.700535,32.492548,-1106.884930,-957.938731,13217.366619,8763.750381
3,2018-02-04,-0.152089,32014.308449,356134,37.32,111.316698,1.252705,38,0.000000,0.000000,...,-0.311040,-0.517416,0.065570,10171.588,9839.902256,28.258612,-1201.875384,-1006.726062,12946.728260,8498.876740
4,2018-02-05,0.102615,63403.182579,533510,37.32,173.767502,1.327510,63,0.000000,0.000000,...,-0.383167,-0.593434,0.069485,9756.687,9312.645482,24.109870,-1361.903399,-1077.761529,13315.272814,7734.331186
5,2018-02-06,-0.006944,100201.500307,785996,29.98,164.269986,1.398123,79,0.023290,-0.024103,...,-0.252539,-0.526184,0.073181,9372.801,9010.735394,30.404847,-1414.951520,-1145.199527,13417.628766,7298.310234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2033,2023-08-27,0.000698,12099.642160,349090,15.08,13.471949,0.300414,45,0.000000,0.000000,...,-0.003368,-0.109584,0.015724,26131.884,26478.236351,26.749939,-904.471454,-800.659263,30906.076489,24321.112511
2034,2023-08-28,0.061116,22692.626550,523057,15.08,16.972956,0.300290,50,0.000000,0.000000,...,-0.000265,-0.110139,0.015718,26138.484,26413.102469,27.164624,-878.947958,-816.317002,30623.324935,24238.822065
2035,2023-08-29,-0.015022,74251.994880,1137083,14.45,21.151151,0.376069,64,0.008473,0.007634,...,0.063722,-0.053439,0.019684,26300.117,26650.054748,52.515235,-721.591055,-797.371813,30370.442488,24305.139512
2036,2023-08-30,-0.049788,35448.258480,690292,13.88,19.220988,0.378955,55,0.001087,-0.002579,...,0.032810,-0.066100,0.019835,26411.117,26768.224793,47.838513,-623.295577,-762.556566,30094.263537,24365.742463


In [773]:
mchiadded

,Open_time,Close,Volume,trades,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,...,Weekly_return,Monthly_return,Volatility,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,Bollinger_Upper,Bollinger_Lower
1,2018-02-02,0.036845,49971.626975,453411,17.31,274.362582,1.302707,66,-0.025423,-0.015115,...,-0.199835,-0.405273,0.068187,10685.637,10431.642876,29.325947,-1072.903152,-920.702181,13526.109039,8882.510961
2,2018-02-03,-0.110343,28725.000735,327519,37.32,146.927769,1.312619,41,-0.046049,-0.012633,...,-0.199377,-0.389094,0.068706,10470.634,10207.700535,32.492548,-1106.884930,-957.938731,13217.366619,8763.750381
3,2018-02-04,-0.152089,32014.308449,356134,37.32,111.316698,1.252705,38,0.000000,0.000000,...,-0.311040,-0.517416,0.065570,10171.588,9839.902256,28.258612,-1201.875384,-1006.726062,12946.728260,8498.876740
4,2018-02-05,0.102615,63403.182579,533510,37.32,173.767502,1.327510,63,0.000000,0.000000,...,-0.383167,-0.593434,0.069485,9756.687,9312.645482,24.109870,-1361.903399,-1077.761529,13315.272814,7734.331186
5,2018-02-06,-0.006944,100201.500307,785996,29.98,164.269986,1.398123,79,0.023290,-0.024103,...,-0.252539,-0.526184,0.073181,9372.801,9010.735394,30.404847,-1414.951520,-1145.199527,13417.628766,7298.310234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2033,2023-08-27,0.000698,12099.642160,349090,15.08,13.471949,0.300414,45,0.000000,0.000000,...,-0.003368,-0.109584,0.015724,26131.884,26478.236351,26.749939,-904.471454,-800.659263,30906.076489,24321.112511
2034,2023-08-28,0.061116,22692.626550,523057,15.08,16.972956,0.300290,50,0.000000,0.000000,...,-0.000265,-0.110139,0.015718,26138.484,26413.102469,27.164624,-878.947958,-816.317002,30623.324935,24238.822065
2035,2023-08-29,-0.015022,74251.994880,1137083,14.45,21.151151,0.376069,64,0.008473,0.007634,...,0.063722,-0.053439,0.019684,26300.117,26650.054748,52.515235,-721.591055,-797.371813,30370.442488,24305.139512
2036,2023-08-30,-0.049788,35448.258480,690292,13.88,19.220988,0.378955,55,0.001087,-0.002579,...,0.032810,-0.066100,0.019835,26411.117,26768.224793,47.838513,-623.295577,-762.556566,30094.263537,24365.742463


In [774]:
mchiadded = mchiadded[:-1]
mchiadded

,Open_time,Close,Volume,trades,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,...,Weekly_return,Monthly_return,Volatility,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,Bollinger_Upper,Bollinger_Lower
1,2018-02-02,0.036845,49971.626975,453411,17.31,274.362582,1.302707,66,-0.025423,-0.015115,...,-0.199835,-0.405273,0.068187,10685.637,10431.642876,29.325947,-1072.903152,-920.702181,13526.109039,8882.510961
2,2018-02-03,-0.110343,28725.000735,327519,37.32,146.927769,1.312619,41,-0.046049,-0.012633,...,-0.199377,-0.389094,0.068706,10470.634,10207.700535,32.492548,-1106.884930,-957.938731,13217.366619,8763.750381
3,2018-02-04,-0.152089,32014.308449,356134,37.32,111.316698,1.252705,38,0.000000,0.000000,...,-0.311040,-0.517416,0.065570,10171.588,9839.902256,28.258612,-1201.875384,-1006.726062,12946.728260,8498.876740
4,2018-02-05,0.102615,63403.182579,533510,37.32,173.767502,1.327510,63,0.000000,0.000000,...,-0.383167,-0.593434,0.069485,9756.687,9312.645482,24.109870,-1361.903399,-1077.761529,13315.272814,7734.331186
5,2018-02-06,-0.006944,100201.500307,785996,29.98,164.269986,1.398123,79,0.023290,-0.024103,...,-0.252539,-0.526184,0.073181,9372.801,9010.735394,30.404847,-1414.951520,-1145.199527,13417.628766,7298.310234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2032,2023-08-26,0.003244,10022.223220,329412,15.08,16.085792,0.300380,46,0.006204,0.013559,...,-0.003166,-0.109689,0.015723,26184.048,26561.895540,24.912057,-922.036622,-774.706216,31054.592756,24483.525244
2033,2023-08-27,0.000698,12099.642160,349090,15.08,13.471949,0.300414,45,0.000000,0.000000,...,-0.003368,-0.109584,0.015724,26131.884,26478.236351,26.749939,-904.471454,-800.659263,30906.076489,24321.112511
2034,2023-08-28,0.061116,22692.626550,523057,15.08,16.972956,0.300290,50,0.000000,0.000000,...,-0.000265,-0.110139,0.015718,26138.484,26413.102469,27.164624,-878.947958,-816.317002,30623.324935,24238.822065
2035,2023-08-29,-0.015022,74251.994880,1137083,14.45,21.151151,0.376069,64,0.008473,0.007634,...,0.063722,-0.053439,0.019684,26300.117,26650.054748,52.515235,-721.591055,-797.371813,30370.442488,24305.139512


In [775]:
missing_values = mchiadded.isna().sum()
print(missing_values)

Open_time                 0
Close                     0
Volume                    0
trades                    0
VIX                       0
Bitcoin_TransactionFee    0
volatility                0
GoogleTrend               0
DowJones                  0
EuroStoxx50               0
Nasdaq                    0
OilWTI                    0
SPX                       0
TradePermin               0
Dominance                 0
Gold                      0
MCHI                      0
Weekly_return             0
Monthly_return            0
Volatility                0
SMA_10                    0
EMA_10                    0
RSI_14                    0
MACD                      0
MACD_Signal               0
Bollinger_Upper           0
Bollinger_Lower           0
dtype: int64


In [776]:
# 각 열에 대해 0 값을 NaN으로 대체하고, NaN 위치를 나중의 0이 아닌 값으로 채우기
for column in mchiadded.columns:
    mchiadded[column] = mchiadded[column].replace(0).fillna(method='bfill')

# 변경된 데이터프레임 확인
mchiadded


/var/folders/8l/d0_bj06x2czf46y6_p5zswgr0000gn/T/ipykernel_28699/1275876703.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mchiadded[column] = mchiadded[column].replace(0).fillna(method='bfill')


,Open_time,Close,Volume,trades,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,...,Weekly_return,Monthly_return,Volatility,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,Bollinger_Upper,Bollinger_Lower
1,2018-02-02,0.036845,49971.626975,453411,17.31,274.362582,1.302707,66,-0.025423,-0.015115,...,-0.199835,-0.405273,0.068187,10685.637,10431.642876,29.325947,-1072.903152,-920.702181,13526.109039,8882.510961
2,2018-02-03,-0.110343,28725.000735,327519,37.32,146.927769,1.312619,41,-0.046049,-0.012633,...,-0.199377,-0.389094,0.068706,10470.634,10207.700535,32.492548,-1106.884930,-957.938731,13217.366619,8763.750381
3,2018-02-04,-0.152089,32014.308449,356134,37.32,111.316698,1.252705,38,-0.046049,-0.012633,...,-0.311040,-0.517416,0.065570,10171.588,9839.902256,28.258612,-1201.875384,-1006.726062,12946.728260,8498.876740
4,2018-02-05,0.102615,63403.182579,533510,37.32,173.767502,1.327510,63,-0.046049,-0.012633,...,-0.383167,-0.593434,0.069485,9756.687,9312.645482,24.109870,-1361.903399,-1077.761529,13315.272814,7734.331186
5,2018-02-06,-0.006944,100201.500307,785996,29.98,164.269986,1.398123,79,0.023290,-0.024103,...,-0.252539,-0.526184,0.073181,9372.801,9010.735394,30.404847,-1414.951520,-1145.199527,13417.628766,7298.310234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2032,2023-08-26,0.003244,10022.223220,329412,15.08,16.085792,0.300380,46,0.006204,0.013559,...,-0.003166,-0.109689,0.015723,26184.048,26561.895540,24.912057,-922.036622,-774.706216,31054.592756,24483.525244
2033,2023-08-27,0.000698,12099.642160,349090,15.08,13.471949,0.300414,45,0.006204,0.013559,...,-0.003368,-0.109584,0.015724,26131.884,26478.236351,26.749939,-904.471454,-800.659263,30906.076489,24321.112511
2034,2023-08-28,0.061116,22692.626550,523057,15.08,16.972956,0.300290,50,0.006204,0.013559,...,-0.000265,-0.110139,0.015718,26138.484,26413.102469,27.164624,-878.947958,-816.317002,30623.324935,24238.822065
2035,2023-08-29,-0.015022,74251.994880,1137083,14.45,21.151151,0.376069,64,0.008473,0.007634,...,0.063722,-0.053439,0.019684,26300.117,26650.054748,52.515235,-721.591055,-797.371813,30370.442488,24305.139512


In [777]:
import numpy as np

# Open_time을 제외한 컬럼들에 대해서만 윈저라이징
for column in mchiadded.columns:
    if column != 'Open_time':
        mean_value = mchiadded[column].mean()
        std_value = mchiadded[column].std()
        
        upper_bound = mean_value + 3 * std_value
        lower_bound = mean_value - 3 * std_value
        
        mchiadded[column] = np.where(mchiadded[column] > upper_bound, upper_bound, mchiadded[column])
        mchiadded[column] = np.where(mchiadded[column] < lower_bound, lower_bound, mchiadded[column])


mchiadded


/var/folders/8l/d0_bj06x2czf46y6_p5zswgr0000gn/T/ipykernel_28699/532633306.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mchiadded[column] = np.where(mchiadded[column] > upper_bound, upper_bound, mchiadded[column])
/var/folders/8l/d0_bj06x2czf46y6_p5zswgr0000gn/T/ipykernel_28699/532633306.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mchiadded[column] = np.where(mchiadded[column] < lower_bound, lower_bound, mchiadded[column])


,Open_time,Close,Volume,trades,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,...,Weekly_return,Monthly_return,Volatility,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,Bollinger_Upper,Bollinger_Lower
1,2018-02-02,0.036845,49971.626975,453411.0,17.31,192.333696,1.302707,66.0,-0.025423,-0.015115,...,-0.199835,-0.405273,0.068187,10685.637,10431.642876,29.325947,-1072.903152,-920.702181,13526.109039,8882.510961
2,2018-02-03,-0.109893,28725.000735,327519.0,37.32,146.927769,1.312619,41.0,-0.041269,-0.012633,...,-0.199377,-0.389094,0.068706,10470.634,10207.700535,32.492548,-1106.884930,-957.938731,13217.366619,8763.750381
3,2018-02-04,-0.109893,32014.308449,356134.0,37.32,111.316698,1.252705,38.0,-0.041269,-0.012633,...,-0.291956,-0.517416,0.065570,10171.588,9839.902256,28.258612,-1201.875384,-1006.726062,12946.728260,8498.876740
4,2018-02-05,0.102615,63403.182579,533510.0,37.32,173.767502,1.327510,63.0,-0.041269,-0.012633,...,-0.291956,-0.593434,0.069485,9756.687,9312.645482,24.109870,-1361.903399,-1077.761529,13315.272814,7734.331186
5,2018-02-06,-0.006944,100201.500307,785996.0,29.98,164.269986,1.398123,79.0,0.023290,-0.024103,...,-0.252539,-0.526184,0.073181,9372.801,9010.735394,30.404847,-1414.951520,-1145.199527,13417.628766,7298.310234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2032,2023-08-26,0.003244,10022.223220,329412.0,15.08,16.085792,0.300380,46.0,0.006204,0.013559,...,-0.003166,-0.109689,0.015723,26184.048,26561.895540,24.912057,-922.036622,-774.706216,31054.592756,24483.525244
2033,2023-08-27,0.000698,12099.642160,349090.0,15.08,13.471949,0.300414,45.0,0.006204,0.013559,...,-0.003368,-0.109584,0.015724,26131.884,26478.236351,26.749939,-904.471454,-800.659263,30906.076489,24321.112511
2034,2023-08-28,0.061116,22692.626550,523057.0,15.08,16.972956,0.300290,50.0,0.006204,0.013559,...,-0.000265,-0.110139,0.015718,26138.484,26413.102469,27.164624,-878.947958,-816.317002,30623.324935,24238.822065
2035,2023-08-29,-0.015022,74251.994880,1137083.0,14.45,21.151151,0.376069,64.0,0.008473,0.007634,...,0.063722,-0.053439,0.019684,26300.117,26650.054748,52.515235,-721.591055,-797.371813,30370.442488,24305.139512


In [778]:
mchiadded.to_csv('Dataset/pctchange_notnormalized_partial.csv', index=False)

Lasso, LightGBM, randomforest


CNN-LSTM

window 하나 들어갔을 때, 그 윈도우 하나 내에서의 함수 짜보기

CNN 따로 짜고, LSTM 따로 짜서 쓸 수 있게 각각 따로 코드 만들어서 짜기